In [9]:
%load_ext autoreload
%autoreload 2
from bokeh.io import push_notebook,show,output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, OpenURL, TapTool, CustomJS,BoxSelectTool,Rect,Ellipse
import numpy as np
from image_vis import *
output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [2]:
import matplotlib.image as mpimg
import scipy
file_name='./chess2.jpg'
#file_name='./data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001.png'
#file_name='./data_nuclis_dev/2tg_E98p_224a001/2tg_E98p_224a001.png'
#file_type=255
file_type=1
img2=mpimg.imread(file_name)*file_type
alpha=np.ones((img2.shape[0],img2.shape[1],1))*255
img2=np.concatenate([img2[:,:,:3],alpha],axis=2)

x_rand=[]
y_rand=[]
probs=[]
#img2=scipy.misc.imresize(img2,(img2.shape[0]/4,img2.shape[1]/4))
N1,N2=img2.shape[:2]
print img2.shape

(650, 650, 4)


In [3]:
red_points=ImagePoints(img2)
green_points=ImagePoints(img2,'#00FF00','#00FF00')
len(red_points.yg)

0

In [4]:
red_points.load(file_name)
len(red_points.yg)

81

In [11]:
red_points.save(file_name)

In [7]:
t2=visualize_image(img2,[red_points,green_points],'red_points')

In [46]:
def dist(x1,x2,y1,y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

def group(values):
    import numpy as np
    values = np.array(values)
    values.sort()
    dif = np.ones(values.shape,values.dtype)
    dif[1:] = np.diff(values)
    idx = np.where(dif>0)
    vals = values[idx]
    count = np.diff(idx)
    return vals

print probs.shape
prob_th=0.5
dist_th=10
clust_th=10
#clust_th=0
x_rand=x_rand[probs>prob_th]
y_rand=y_rand[probs>prob_th]
probs=probs[probs>prob_th]
probs[probs>prob_th].shape,probs.shape
clusters=np.ones((probs.shape[0],))*-1                  
for i in range(probs.shape[0]):
#for i in range(1):
    dists=dist(x_rand,x_rand[i],y_rand,y_rand[i])
    #print "dists",dists
    dist_filter=dists<=dist_th
    #print "dist_filter",np.where(dist_filter)
    without_cluster=np.logical_and(dist_filter,clusters==-1)
    clusters[without_cluster]=i
    different_clusters=group(clusters[dist_filter])
    for c in different_clusters:
        clusters[clusters==c]=i
clust=group(clusters)
x_rand2=[]
y_rand2=[]
for c in clust:
    #print clusters[clusters==c].shape[0]
    if clusters[clusters==c].shape[0]>clust_th:
        x_rand2.append(np.mean(x_rand[clusters==c]))
        y_rand2.append(np.mean(y_rand[clusters==c]))
x_rand=np.array(x_rand2)
y_rand=np.array(y_rand2)

(4041,)


In [107]:
x_rand.shape

(81,)

In [88]:
import numpy as np
np.random.randint(0,2,size=(3,3))

array([[0, 0, 1],
       [0, 0, 0],
       [1, 1, 0]])

In [6]:
generate_random_points_with_probs(red_points,green_points,M=10000,percent_negative_filter=5)
#probs

[False False False ..., False False False]
[22 19 91 ..., 81 79 93]
0.593474426808
1134
(1134,)


In [13]:
X,Y=dataset_from_points(green_points)


(1134, 40, 40, 3) (1134, 2)


In [15]:
batch_size = 200 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 10 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.5 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 3000 # the FC layer will have 512 neurons

In [16]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import keras
import numpy as np
inp = Input(shape=(40, 40, 3)) # depth goes last in TensorFlow back-end (first in Theano)
#inp = Input(shape=(612, 816, 3)) # depth goes last in TensorFlow back-end (first in Theano)
#inp = Input(shape=(500, 500, 3)) # depth goes last in TensorFlow back-end (first in Theano)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax

'''flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(2, activation='softmax')(drop_3)'''
#conv_5= Convolution2D(30,(10,10),strides=(1,1),padding='same',activation='relu')(drop_2)
#Per nuclis millor 100
conv_5= Convolution2D(100,(10,10),strides=(1,1),padding='valid',activation='relu')(drop_2)
drop_3= Dropout(drop_prob_1)(conv_5)
conv_6=Convolution2D(2,(1,1),padding='valid',activation='softmax')(drop_3)
out=Flatten()(conv_6)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer=adam, # using the Adam optimiser
              metrics=['accuracy']) 
model.summary()
model.layers

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 40, 40, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 32)        9632      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 40, 32)        102432    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 64)        204864    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 64)        409664    
__________

In [21]:
model.fit(X, Y,                # Train the model using the training set...
          batch_size=batch_size, epochs=10,
          verbose=1, validation_split=0.1) 

Train on 7149 samples, validate on 795 samples
Epoch 1/10
7149/7149 [==============================] - 1s - loss: 0.1398 - acc: 0.9487 - val_loss: 0.1245 - val_acc: 0.9560
Epoch 2/10
7149/7149 [==============================] - 1s - loss: 0.1424 - acc: 0.9452 - val_loss: 0.1285 - val_acc: 0.9509
Epoch 3/10
7149/7149 [==============================] - 1s - loss: 0.1398 - acc: 0.9477 - val_loss: 0.1285 - val_acc: 0.9484
Epoch 4/10
7149/7149 [==============================] - 1s - loss: 0.1369 - acc: 0.9496 - val_loss: 0.1242 - val_acc: 0.9434
Epoch 5/10
7149/7149 [==============================] - 1s - loss: 0.1322 - acc: 0.9492 - val_loss: 0.1277 - val_acc: 0.9522
Epoch 6/10
7149/7149 [==============================] - 1s - loss: 0.1282 - acc: 0.9505 - val_loss: 0.1271 - val_acc: 0.9484
Epoch 7/10
7149/7149 [==============================] - 1s - loss: 0.1310 - acc: 0.9503 - val_loss: 0.1194 - val_acc: 0.9560
Epoch 8/10
7149/7149 [==============================] - 1s - loss: 0.1317 - ac

In [22]:
model.save_weights('conv_nuclis.h5')

In [137]:
model.load_weights('conv_chess.h5')

In [45]:
import coremltools
converter=coremltools.converters.keras.convert(model,input_names=['chess_image'],image_input_names=['chess_image'])
converter.save('chess500x500.mlmodel')

0 : input_12, <keras.engine.topology.InputLayer object at 0x7f1eb162e410>
1 : conv2d_67, <keras.layers.convolutional.Conv2D object at 0x7f1eb162e450>
2 : conv2d_67__activation__, <keras.layers.core.Activation object at 0x7f1eb142fc90>
3 : conv2d_68, <keras.layers.convolutional.Conv2D object at 0x7f1e85374290>
4 : conv2d_68__activation__, <keras.layers.core.Activation object at 0x7f1eb142fbd0>
5 : max_pooling2d_23, <keras.layers.pooling.MaxPooling2D object at 0x7f1e84b63f90>
6 : conv2d_69, <keras.layers.convolutional.Conv2D object at 0x7f1e854ed9d0>
7 : conv2d_69__activation__, <keras.layers.core.Activation object at 0x7f1eb142ff90>
8 : conv2d_70, <keras.layers.convolutional.Conv2D object at 0x7f1e854ed2d0>
9 : conv2d_70__activation__, <keras.layers.core.Activation object at 0x7f1eb142ffd0>
10 : max_pooling2d_24, <keras.layers.pooling.MaxPooling2D object at 0x7f1eb162e810>
11 : conv2d_71, <keras.layers.convolutional.Conv2D object at 0x7f1eb15af3d0>
12 : conv2d_71__activation__, <keras.l

In [48]:
model.evaluate(X,Y)

3904/4041 [===========================>..] - ETA: 0s

[0.19446848313338211, 0.92873051239694293]

In [44]:
probs=model.predict(X,verbose=1)[:,0]>0.5

4041/4041 [==============================] - 0s     


In [169]:
conv_out_y=125
conv_out_x=125
probs=model.predict((img2[:,:,:3]/255.).reshape(1,N1,N2,3)).reshape(conv_out_y,conv_out_x,2)[:,:,0].reshape(-1)>0.95

x_rand=[]
y_rand=[]
for k in range(conv_out_y):
    for u in range(conv_out_x):
        y_rand.append(u*4+4)
        x_rand.append(k*4+4)
x_rand=np.array(x_rand)
y_rand=np.array(y_rand)
'''x_cond=np.logical_and(x_rand>=0,x_rand<=499)
y_cond=np.logical_and(y_rand>=0,y_rand<=499)
cond=np.logical_and(x_cond,y_cond)
probs=probs[cond]
x_rand=x_rand[cond]
y_rand=y_rand[cond]'''


ValueError: cannot reshape array of size 3932160 into shape (1,500,500,3)

In [14]:
i=12
im_total=np.zeros((400,400,4))
p4=figure(x_range=(0,400), y_range=(0,400),plot_width=400,plot_height=400)#,tools=[box_select])
for i in range(10):
    for j in range(10):        
        im=(np.concatenate([X[i*10+j],np.ones((40,40,1),dtype=np.float64)],axis=2)*255)
        im_total[i*40:(i+1)*40,j*40:(j+1)*40,:]=im
        
        
print im.shape
p4.image_rgba([im_total.astype(np.uint8)],x=0, y=0, dw=400, dh=400)
for i in range(10):
    for j in range(10): 
        yi=Y[i*10+j,0]
        #print y
        p4.ellipse(x=j*40+20,
            y=i*40+20,
            width=10,
            height=10,
            line_alpha=yi,
            fill_alpha=yi,
            line_color='#00FF00',
            fill_color='#00FF00')
t2=show(p4,notebook_handle=True)
y_rand[0],x_rand[0]




(40, 40, 4)


IndexError: list index out of range